# Vantage6

In [ ]:
import time
from vantage6.client import Client
from sklearn.linear_model import LogisticRegression
from src import config

## Reseacher role

We are now _sharing_ the OPC-Radiomics data in vantage6 data nodes and, as a researcher in a collaboration, you can send tasks to them. We will send a logistic regression task that does exactlly the same as we did in the previous tutorial. The difference is that all the steps are now automated.

Before starting, you need to edit the `config.py` file, which is located in the `src` directory. The information you need to provide in this file will be shared with you during the lecture.

In [ ]:
# Initialize the vantage6 client object, and run the authentication
client = Client(
    config.server_url, config.server_port, config.server_api,
    verbose=True
)
client.authenticate(config.username, config.password)
client.setup_encryption(None)

In [ ]:
# EXERCISE
# Vantage6 logistic regression task input, set the maximum iterations to a high value and delta to a low value
input_ = {
    'method': 'master',
    'master': True,
    'kwargs': {
        'org_ids': [2, 3],
        'predictors': <YOUR_ANSWER_HERE>,
        'outcome': <YOUR_ANSWER_HERE>,
        'classes': <YOUR_ANSWER_HERE>,
        'max_iter': <YOUR_ANSWER_HERE>,
        'delta': <YOUR_ANSWER_HERE>,
    }
}

In [ ]:
# Vantage6 logistic regression task creation
task = client.task.create(
    collaboration=1,
    organizations=[2, 3],
    name='v6-logistic-regression-py',
    image='ghcr.io/maastrichtu-cds/v6-logistic-regression-py:latest',
    description='logistic regression',
    input=input_,
    data_format='json'
)

In [ ]:
%time
# Retrieving results
task_info = client.task.get(task['id'], include_results=True)
while not task_info.get('complete'):
    print('Waiting for results...')
    task_info = client.task.get(task['id'], include_results=True)
    time.sleep(1)
result_info = client.result.list(task=task_info['id'])
results = result_info['data'][0]['result']

In [ ]:
# EXERCISE
# Explore the results dictionary


- How do the result compare with the one you did manually?

In [ ]:
# EXERCISE
# Fix the max number of iterations to 5 and send a new task


- What happened now?

## References

- [Vantage6 documentation](https://docs.vantage6.ai/en/main/)